# Stacking ensembling

Vamos a crear un ensamble de tipo stacking. Para ello se utilizaron las siguientes fuentes:
 - https://mlwave.com/kaggle-ensembling-guide/
 - http://blog.kaggle.com/2016/12/27/a-kagglers-guide-to-model-stacking-in-practice/
 - https://github.com/emanuele/kaggle_pbr/blob/master/blend.py

Y se desarrollaron las siguientes secciones:
 - [Predictores base](#Predictores-base)
     - [Xgboost](#Xgboost)
     - [Random Forest](#Random-Forest)
     - [AdaBoost](#AdaBoost)
 - [Metafeatures](#Metafeatures)
 - [Predictor Stacking](#Predictor-Stacking)

In [1]:
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from time import time
import xgboost as xgb
import pandas as pd
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))
predicciones_csv = os.path.join(current_folder, '..', 'predictions.csv')

Cargo el df con los features.

In [3]:
pd.options.mode.chained_assignment = None
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [4]:
assert(df_features.shape[0] == get_clean_df()['person'].unique().shape[0])

Cargo el set de entrenamiento.

In [5]:
%run $set_de_entrenamiento_testing_y_prediccion

labels_with_features = labels.merge(df_features, how='inner', on='person')
train = labels_with_features.drop('label', axis=1)
train_target = labels_with_features['label']

### Predictores base

En esta sección vamos a preparar los predictores base a utilizar. Estos son los mismos que se encuentran en la carpeta *Algoritmos de ML*.

In [6]:
base_predictors = []

#### Xgboost

Nota: vamos a usar XGBRegressor para tener la misma interfaz con el resto de los predictores. Los hiperparámetros tienen distinto nombre, pero producen los mismos resultados. Se puede consultar la documentación de xgboost para encontrar los nombres de los parámetros: https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor. 

In [7]:
param = {
    'objective': 'reg:logistic',
    'colsample_bylevel': 0.6605668347627213,
    'colsample_bytree': 0.5279014819087092,
    'min_child_weight': 4.302125582335056,
    'learning_rate': 0.15803667962605694,
    'max_delta_step': 7.592652591386328,
    'n_estimators': 65,
    'reg_lambda': 1.1181195507921775,
    'max_depth': 9,
    'silent': True,
    'subsample': 0.43744176565530823,
    'reg_alpha': 3.845311207046479,
    'gamma': 6.219264874528072
}

base_predictors.append(XGBRegressor(**param))

#### Random Forest

In [8]:
param = {
    'bootstrap': True,
    'max_depth': 10,
    'max_features': 81,
    'min_samples_leaf': 49,
    'min_samples_split': 8,
    'n_estimators': 56
}

base_predictors.append(RandomForestRegressor(**param))

#### AdaBoost

In [9]:
param = {
    'n_estimators': 128,
    'loss': 'linear',
    'learning_rate': 0.07,
    'base_estimator': DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')
}

base_predictors.append(AdaBoostRegressor(**param))

#### DecisionTree

In [10]:
base_predictors.append(DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best'))

### Metafeatures

Ahora vamos a realizar una predicción por cada predictor base, y las vamos a agregar a una copia del set de entrenamiento (*train_meta*).

In [11]:
train_meta = train.copy()
for predictor in base_predictors:
    train_meta[predictor.__class__.__name__] = np.nan

Para realizar las predicciones utilizar cross validation.

In [12]:
%%time
kf = KFold(n_splits=10, shuffle=False)
for train_i, validation_i in kf.split(train):    
    for predictor in base_predictors:
        # como warm_start=False cada vez que llamo fit, el modelo se reinicia
        predictor.fit(train.iloc[train_i], train_target.iloc[train_i]) # train
        train_meta[predictor.__class__.__name__].iloc[validation_i] = predictor.predict(train.iloc[validation_i]) # predict

/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if g

CPU times: user 8min 45s, sys: 198 ms, total: 8min 46s
Wall time: 8min 46s


In [13]:
train_meta.head(3)

,screen_resolution_height mean,screen_resolution_width mean,screen_resolution_height std,screen_resolution_width std,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,...,Sunday,Thursday,Tuesday,Wednesday,madrugada,maniana,tarde,noche,XGBRegressor,RandomForestRegressor,AdaBoostRegressor,DecisionTreeRegressor
person,,,,,,,,,,,,,,,,,,,,,,,,,
0566e9c1,568.0,320.0,0.0,0.0,6.0,3.0,1.0,1.0,15.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0,0,1,0,0.018380,0.021013,0.127402,0.007599
6ec7ee77,640.0,360.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.079287,0.090634,0.308829,0.117647
abe7a2fb,640.0,360.0,0.0,0.0,9.0,14.0,1.0,0.0,9.0,0.0,4.0,6.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.017295,0.030771,0.195528,0.007599


### Predictor Stacking

Ahora vamos a entrenar un nuevo modelo utilizando como features las predicciones anteriores (metafeatures). También podemos agregar algunos de los features originales.

Para esto realizamos un Random Search. TODO 

In [14]:
param = {
    'learning_rate': 0.3,
    'gamma': 1.6317896840572566,
    'max_depth': 2,
    'objective': 'reg:logistic',
    'n_estimators': 25
}

stacking = XGBRegressor(**param)
stack_train = train_meta[[predictor.__class__.__name__ for predictor in base_predictors]]

In [15]:
%%time
scores = cross_val_score(stacking, stack_train, train_target, cv=10, scoring='roc_auc')
print("Accuracy: %0.6f (+/- %0.6f)" % (scores.mean(), scores.std() * 2))

/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if g

Accuracy: 0.876813 (+/- 0.019586)
CPU times: user 1.76 s, sys: 3.97 ms, total: 1.77 s
Wall time: 1.77 s


In [16]:
stacking.fit(stack_train, train_target)

/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=1.6317896840572566,
             importance_type='gain', learning_rate=0.3, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=25,
             n_jobs=1, nthread=None, objective='reg:logistic', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

### Bayesian Optimization

Búsque de hiperparámetros para el stacking.

In [17]:
pbounds = {
    'max_depth': (2, 30),
    'eta': (0, 1),
    'gamma': (0, 20),
    'min_child_weight': (1, 8),
    'max_delta_step': (1, 8),
    'subsample': (0, 1),
    'colsample_bytree': (0, 1),
    'colsample_bylevel': (0, 1),
    'lambda': (1, 10),
    'alpha': (0, 8)
}

discrete = ['max_depth'] # parámetros discretos
cv_splits = 10 # cantidad de splits en el cv
num_round = 100 # cantidad máxima de boosts

In [18]:
dtrain = xgb.DMatrix(stack_train, label=train_target)
def cv_score_xgb(**param):
    param['silent'] = 1
    param['objective'] = 'reg:logistic'
    
    # transformo los valores que deben ser discretos
    for d in discrete:
        param[d] = int(param[d])
    
    # hago el cv
    scores = xgb.cv(param, dtrain, nfold=cv_splits, metrics='auc', verbose_eval=False, shuffle=False, stratified=False, num_boost_round=num_round, early_stopping_rounds=20)
    return scores['test-auc-mean'].max()

In [19]:
%%time
optimizer = BayesianOptimization(f=cv_score_xgb, pbounds=pbounds)
# optimizer.probe(
#     params = {'eta': 0.06027, 'gamma': 8.548, 'max_depth': 16}
# )
optimizer.maximize(
    init_points=20,
    n_iter=20,
)

|   iter    |  target   |   alpha   | colsam... | colsam... |    eta    |   gamma   |  lambda   | max_de... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.8765   |  6.385    |  0.7464   |  0.8593   |  0.2693   |  7.562    |  7.709    |  1.445    |  7.232    |  5.316    |  0.6595   |
|  2        |  0.875    |  7.262    |  0.2786   |  0.6384   |  0.6078   |  6.138    |  4.342    |  1.4      |  28.56    |  5.768    |  0.6861   |
|  3        |  0.8702   |  4.797    |  0.2917   |  0.5098   |  0.9009   |  10.39    |  9.394    |  4.886    |  2.86     |  4.015    |  0.6473   |
|  4        |  0.8694   |  3.512    |  0.5126   |  0.35     |  0.4883   |  15.98    |  2.699    |  3.516    |  19.13    |  1.381    |  0.2142   |
|  5        |  0.8733   |  1.071    |  0.2793   |  0.783    |  0.8484   |  14.38    |  9.868    |  3.853    |  17.82    |  5

|  34       |  0.8755   |  7.881    |  0.9296   |  0.4309   |  0.4312   |  1.829    |  9.516    |  7.219    |  2.102    |  7.89     |  0.9927   |
|  35       |  0.8754   |  0.2149   |  0.4656   |  0.9507   |  0.5994   |  8.229    |  9.973    |  7.79     |  28.68    |  1.059    |  0.5013   |
|  36       |  0.8744   |  0.01863  |  0.561    |  0.9656   |  0.7457   |  13.44    |  2.29     |  7.794    |  14.98    |  1.154    |  0.8877   |
|  37       |  0.8764   |  7.576    |  0.7308   |  0.6384   |  0.9376   |  3.508    |  4.275    |  7.965    |  29.56    |  2.494    |  0.8949   |
|  38       |  0.8497   |  7.978    |  0.0      |  1.0      |  2.346e-0 |  6.393    |  3.449    |  8.0      |  30.0     |  8.0      |  1.0      |
|  39       |  0.871    |  0.9893   |  0.8958   |  0.8072   |  0.9902   |  17.25    |  8.943    |  1.588    |  28.38    |  1.21     |  0.5133   |
|  40       |  0.8703   |  5.221    |  0.861    |  0.9053   |  0.9827   |  0.8967   |  9.686    |  7.753    |  19.19    |  3

In [20]:
optimizer.max

{'target': 0.8770772000000001,
 'params': {'alpha': 3.589060371439908,
  'colsample_bylevel': 0.9743874410279757,
  'colsample_bytree': 0.9792712927886779,
  'eta': 0.7653596968199986,
  'gamma': 0.21721627657683262,
  'lambda': 8.475874378255888,
  'max_delta_step': 1.120340073446381,
  'max_depth': 2.390290920333375,
  'min_child_weight': 7.217759618943042,
  'subsample': 0.9168917646461506}}

Escribo el mejor resultado en un archivo

In [21]:
hyperparameter_data = {
    'algorithm': 'stacking',
    'hyperparameters': optimizer.max['params'],
    'cv_splits': cv_splits,
    'auc': optimizer.max['target'],
    'features': train.columns
}

In [22]:
%run -i write_hyperparameters.py

### Predicción del set testing con los modelos base

Utilizamos los modelos entrenados con el 100% del set de entrenamiento para predecir el set de testing.

In [23]:
testing = labels_to_predict.merge(df_features, how='inner', on='person')
assert(testing.shape[0] == labels_to_predict.shape[0])

In [24]:
testing_with_base_predictions = testing.copy()
for predictor in base_predictors:
    testing_with_base_predictions[predictor.__class__.__name__] = np.nan

In [25]:
%%time
for predictor in base_predictors:
    # como warm_start=False cada vez que llamo fit, el modelo se reinicia
    predictor.fit(train, train_target) # train
    testing_with_base_predictions[predictor.__class__.__name__] = predictor.predict(testing) # predict

/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/sebas/.envs/trocafone/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


CPU times: user 1min 12s, sys: 167 ms, total: 1min 12s
Wall time: 1min 13s


### Predicción del set testing con el stacking

Utilizamos las predicciones como features para que el Stacking las combine en una prediccón final.

In [26]:
testing_with_base_predictions_for_stacking = testing_with_base_predictions[[predictor.__class__.__name__ for predictor in base_predictors]]
testing_with_base_predictions_for_stacking.head(3)

,XGBRegressor,RandomForestRegressor,AdaBoostRegressor,DecisionTreeRegressor
person,,,,
4886f805,0.005293,0.001813,0.037285,0.007434
0297fc1e,0.017091,0.057994,0.139586,0.039040
2d681dd8,0.009014,0.002503,0.039510,0.007434


In [27]:
predictions = stacking.predict(testing_with_base_predictions_for_stacking)

In [28]:
testing_target = pd.DataFrame(data=stacking.predict(testing_with_base_predictions_for_stacking))
testing_target.index = testing_with_base_predictions_for_stacking.index
testing_target.index.name = 'person'
testing_target.columns = ['label']

In [29]:
testing_target.to_csv(predicciones_csv)